In [1]:

%run ../../load_magic/storage.py
%run ../../load_magic/dataframes.py
%run ../../load_magic/environment.py
%pprint
notebook_path = get_notebook_path()
print(notebook_path)
s = Storage()
print([fn for fn in dir(s) if not fn.startswith('_')])
%who

Pretty printing has been turned OFF
C:\Users\dev\Documents\repositories\notebooks\StatsByCountry\ipynb\Rule of Law Stats.ipynb
data_folder: ../data/
saves_folder: ../saves/
['attempt_to_pickle', 'data_csv_folder', 'data_folder', 'encoding_type', 'load_csv', 'load_dataframes', 'load_object', 'save_dataframes', 'saves_csv_folder', 'saves_folder', 'saves_pickle_folder', 'store_objects']
Config	 Storage	 csv	 example_iterrows	 get_column_descriptions	 get_max_rsquared_adj	 get_module_version	 get_notebook_path	 get_page_tables	 
ipykernel	 json	 jupyter_config_dir	 math	 notebook_path	 notebookapp	 os	 pd	 pickle	 
plt	 s	 show_linear_scatterplot	 sm	 sns	 stats	 sys	 urllib	 


In [ ]:

# https://databank.worldbank.org/source/worldwide-governance-indicators
csv_path = os.path.join(DATA_CSV_FOLDER, 'Data.csv')
governance_df = pd.read_csv(csv_path, encoding=ENCODING_TYPE, low_memory=False)
governance_df.columns = ['Country', 'Country_Code', 'Series_Name', 'Series_Code', 1996, 1998, 2000, 2002, 2003, 2004, 2005, 2006, 2007, 2008,
                         2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018]

In [ ]:

match_series = (governance_df['Country'] == 'Iran, Islamic Rep.')
governance_df.loc[match_series, 'Country'] = 'Iran'

In [ ]:

match_series = (governance_df['Country'] == 'Egypt, Arab Rep.')
governance_df.loc[match_series, 'Country'] = 'Egypt'

In [ ]:

match_series = (governance_df['Country'] == 'Venezuela, RB')
governance_df.loc[match_series, 'Country'] = 'Venezuela'

In [ ]:

match_series = (governance_df['Country'] == 'Syrian Arab Republic')
governance_df.loc[match_series, 'Country'] = 'Syria'

In [ ]:

match_series = (governance_df['Country'] == 'Yemen, Rep.')
governance_df.loc[match_series, 'Country'] = 'Yemen'

In [ ]:

governance_df.set_index('Country', inplace=True)

In [ ]:

country = 'Nigeria'
year = 2015
match_series = (governance_df.index == country)
df = governance_df[match_series][[year]]
df.index = governance_df[match_series]['Series_Code']
df.sample(5).to_dict()[year]

In [ ]:

gun_stats_df = load_object('gun_stats_df')

In [ ]:

gun_stats_df.sample(8).T

In [ ]:

set(governance_df.index.unique().tolist()) - set(gun_stats_df.index.unique().tolist())

In [ ]:

set(gun_stats_df.index.unique().tolist()) - set(governance_df.index.unique().tolist())

In [ ]:

%run ../../load_magic/lists.py

In [ ]:

typos_df = check_for_typos(left_list=governance_df.index.unique().tolist(), right_list=gun_stats_df.index.unique().tolist())
match_series = (typos_df['max_similarity'] < 1.0)
typos_df[match_series].sort_values('max_similarity', ascending=False).head(10)

In [ ]:

sc_list = ['_'.join(sc.split('.')) for sc in governance_df['Series_Code'].unique().tolist() if sc is not pd.np.nan]
merge_df = pd.concat(
    [
        gun_stats_df,
        pd.DataFrame(
            [[pd.np.nan] * len(sc_list)], 
            index=gun_stats_df.index, 
            columns=sc_list
        )
    ], axis=1
)
for country, row_series in merge_df.iterrows():
    year = row_series['Year_listed']
    match_series = (governance_df.index == country)
    df = governance_df[match_series][[year]]
    if df.shape[0] > 0:
        df.index = governance_df[match_series]['Series_Code']

        # Get input row in dictionary format; key = col_name
        row_dict = df.to_dict()[year]

        for column_name in sc_list:
            merge_df.loc[country, column_name] = row_dict['.'.join(column_name.split('_'))]
        
merge_df.sample(n=5).T.sample(n=8).T

In [ ]:

[fn for fn in dir(governance_df) if ('dup' in fn.lower()) or ('unique' in fn.lower())]

In [ ]:

[fn for fn in dir(governance_df) if ('index' in fn.lower())]

In [ ]:

capitalism_gini_df = load_object('capitalism_gini_df')
capitalism_gini_df.columns.tolist()

In [ ]:

csv_path = os.path.join(DATA_CSV_FOLDER, 'income_df.csv')
income_df = pd.read_csv(csv_path, encoding=ENCODING_TYPE, low_memory=False)
match_series = (income_df['Country'] == 'Central African Rep.')
income_df.loc[match_series, 'Country'] = 'Central African Republic'
match_series = (income_df['Country'] == 'Dominican Rep.')
income_df.loc[match_series, 'Country'] = 'Dominican Republic'
match_series = (income_df['Country'] == 'Czech Rep.')
income_df.loc[match_series, 'Country'] = 'Czech Republic'
income_df.set_index('Country', inplace=True)
income_df.columns = [int(cn) for cn in income_df.columns.tolist()]

In [ ]:

set(merge_df.index.unique().tolist()) - set(income_df.index.unique().tolist())

In [ ]:

set(income_df.index.unique().tolist()) - set(merge_df.index.unique().tolist())

In [ ]:

typos_df = check_for_typos(left_list=merge_df.index.unique().tolist(), right_list=income_df.index.unique().tolist())
match_series = (typos_df['max_similarity'] < 1.0)
typos_df[match_series].sort_values('max_similarity', ascending=False).head(10)

In [ ]:

income_df.sample(n=5).T.sample(n=8).T

In [ ]:

merge_df.columns.tolist()

In [ ]:

year = 1982
country = 'Uganda'
income_df.loc[country, year]

In [ ]:

gdp_df.loc[country, 'GDP_per_capita'] = income_df.loc[country, year]
gdp_df.loc[country, 'GDP_per_capita']

In [ ]:

gdp_df = pd.concat(
    [
        merge_df,
        pd.DataFrame(
            [], 
            index=merge_df.index, 
            columns=['GDP_per_capita']
        )
    ], axis=1
)
for country, row_series in merge_df.iterrows():
    year = row_series['Year_listed']
    if country in income_df.index:
        gdp_df.loc[country, 'GDP_per_capita'] = income_df.loc[country, year]
        
gdp_df['GDP_per_capita'].unique().tolist()

In [ ]:

columns_list = [cn for cn in governance_df['Series_Name'].unique().tolist() if 'Rule of Law' in str(cn)]
match_series = (governance_df['Series_Name'].isin(columns_list))
columns_list = ['Series_Name', 'Series_Code']
values_list = governance_df[match_series][columns_list].drop_duplicates().reset_index(drop=True).values
RoL_dict = {item[0]:'_'.join(item[1].split('.')) for item in values_list}

In [ ]:

import matplotlib.pyplot as plt
import seaborn as sns

explanatory_variable = 'Rule of Law: Percentile Rank'
ev_column_name = RoL_dict[explanatory_variable]
response_variable = 'GDP per Capita'
rv_column_name = 'GDP_per_capita'
fc_colunm_name = 'Region_code'
ev_column_name

In [ ]:

columns_list = [ev_column_name, rv_column_name]
gdp_df[columns_list].sample(n=5)

In [ ]:

[idx for idx in gdp_df.index if 'United' in idx]

In [ ]:

[cn for cn in gdp_df.columns if 'region' in cn.lower()]

In [ ]:

sorted(gdp_df['Subregion_code'].unique().tolist())

In [ ]:

import matplotlib.colors as mcolors

bc_list = list(mcolors.BASE_COLORS.keys())
[bc_list[rc] for rc in sorted(gdp_df['Region_code'].unique().tolist())]

In [ ]:

%matplotlib inline
fig1_fig = plt.figure(figsize=(12,8))
columns_list = [ev_column_name, rv_column_name, fc_colunm_name]
df = gdp_df[columns_list].dropna()
scatter_kws = dict(facecolors=[bc_list[rc] for rc in df[fc_colunm_name].tolist()])

# First order (linear) scatterplot
merge_axes_subplot = sns.regplot(x=ev_column_name, y=rv_column_name,
                                 scatter=True, data=df, scatter_kws=scatter_kws)
xlabel_text = plt.xlabel('{} (explanatory variable)'.format(explanatory_variable))
ylabel_text = plt.ylabel('{} (response variable)'.format(response_variable))
kwargs = dict(textcoords='offset points', ha='left', va='bottom',
              bbox=dict(boxstyle='round,pad=0.5', fc='yellow', alpha=0.5),
              arrowprops=dict(arrowstyle='->', connectionstyle='arc3,rad=0'))
most_lawful = df[ev_column_name].max()
most_lawless = df[ev_column_name].min()
poorest = df[rv_column_name].min()
richest = df[rv_column_name].max()
for label, x, y in zip(df.index, df[ev_column_name], df[rv_column_name]):
    if (x == most_lawful):
        annotation = plt.annotate('{} (most lawful)'.format(label), xy=(x, y), xytext=(-85, -40), **kwargs)
    elif (x == most_lawless):
        annotation = plt.annotate('{} (most lawless)'.format(label), xy=(x, y), xytext=(0, 90), **kwargs)
    elif (y == poorest):
        annotation = plt.annotate('{} (poorest)'.format(label), xy=(x, y), xytext=(0, -50), **kwargs)
    elif (y == richest):
        annotation = plt.annotate('{} (richest)'.format(label), xy=(x, y), xytext=(0, -50), **kwargs)
    elif (label == 'United States'):
        annotation = plt.annotate('{} (most evil)'.format(label), xy=(x, y), xytext=(-90, 40), **kwargs)

In [ ]:

def show_linear_scatterplot(df, basecolor_list, ev_column_name, rv_column_name, fc_colunm_name,
                            explanatory_variable, response_variable,
                            ev_min_str='most lawless', ev_max_str='most lawful', rv_min_str='poorest', rv_max_str='richest',
                            ev_min_xytext=(0, 90), ev_max_xytext=(-85, -40), rv_min_xytext=(0, -50), rv_max_xytext=(0, -50),
                            us_xytext=(-90, 40)):
    fig1_fig = plt.figure(figsize=(12,8))
    columns_list = [ev_column_name, rv_column_name, fc_colunm_name]
    df = df[columns_list].dropna()
    scatter_kws = dict(facecolors=[basecolor_list[rc] for rc in df[fc_colunm_name].tolist()])

    # First order (linear) scatterplot
    merge_axes_subplot = sns.regplot(x=ev_column_name, y=rv_column_name,
                                     scatter=True, data=df, scatter_kws=scatter_kws)
    xlabel_text = plt.xlabel('{} (explanatory variable)'.format(explanatory_variable))
    ylabel_text = plt.ylabel('{} (response variable)'.format(response_variable))
    kwargs = dict(textcoords='offset points', ha='left', va='bottom',
                  bbox=dict(boxstyle='round,pad=0.5', fc='yellow', alpha=0.5),
                  arrowprops=dict(arrowstyle='->', connectionstyle='arc3,rad=0'))
    ev_max = df[ev_column_name].max()
    ev_min = df[ev_column_name].min()
    rv_min = df[rv_column_name].min()
    rv_max = df[rv_column_name].max()
    for label, x, y in zip(df.index, df[ev_column_name], df[rv_column_name]):
        if (x == ev_min):
            annotation = plt.annotate('{} ({})'.format(label, ev_min_str), xy=(x, y), xytext=ev_min_xytext, **kwargs)
        elif (x == ev_max):
            annotation = plt.annotate('{} ({})'.format(label, ev_max_str), xy=(x, y), xytext=ev_max_xytext, **kwargs)
        elif (y == rv_min):
            annotation = plt.annotate('{} ({})'.format(label, rv_min_str), xy=(x, y), xytext=rv_min_xytext, **kwargs)
        elif (y == rv_max):
            annotation = plt.annotate('{} ({})'.format(label, rv_max_str), xy=(x, y), xytext=rv_max_xytext, **kwargs)
        elif (label == 'United States'):
            annotation = plt.annotate('{} (most evil)'.format(label), xy=(x, y), xytext=us_xytext, **kwargs)
show_linear_scatterplot(df=gdp_df, basecolor_list=bc_list, ev_column_name='RL_PER_RNK', rv_column_name='GDP_per_capita',
                        fc_colunm_name='Region_code', explanatory_variable='Rule of Law: Percentile Rank',
                        response_variable='GDP per Capita', ev_min_str='most lawless', ev_max_str='most lawful',
                        rv_min_str='poorest', rv_max_str='richest', ev_min_xytext=(0, 90), ev_max_xytext=(-85, -40),
                        rv_min_xytext=(0, -50), rv_max_xytext=(0, -50), us_xytext=(-90, 40))

In [ ]:

show_linear_scatterplot(df=gdp_df, basecolor_list=bc_list, ev_column_name='RL_PER_RNK', rv_column_name='Guns_per_100_residents',
                        fc_colunm_name='Region_code', explanatory_variable='Rule of Law: Percentile Rank',
                        response_variable='Guns per 100 Residents', ev_min_str='most lawless', ev_max_str='most lawful',
                        rv_min_str='least gun nutty', rv_max_str='most gun nutty', ev_min_xytext=(5, 50), ev_max_xytext=(-135, -20),
                        rv_min_xytext=(0, -20), rv_max_xytext=(-100, -50), us_xytext=(-90, 40))

In [ ]:

df = gdp_df.copy()
basecolor_list = bc_list
ev_column_name = 'RL_PER_RNK'
rv_column_name = 'Guns_per_100_residents'
fc_colunm_name = 'Region_code'
explanatory_variable = 'Rule of Law: Percentile Rank'
response_variable = 'Guns per 100 Residents'
ev_min_str = 'most lawless'
ev_max_str = 'most lawful'
rv_min_str = 'least gun nutty'
rv_max_str = 'most gun nutty'
ev_min_xytext = (5, 50)
ev_max_xytext = (-135, -20)
rv_min_xytext = (0, -20)
rv_max_xytext = (-100, -50)
us_xytext = (-90, 40)

fig1_fig = plt.figure(figsize=(12,8))
columns_list = [ev_column_name, rv_column_name, fc_colunm_name]
df = df[columns_list].dropna()
scatter_kws = dict(facecolors=[basecolor_list[rc] for rc in df[fc_colunm_name].tolist()])

# First order (linear) scatterplot
merge_axes_subplot = sns.regplot(x=ev_column_name, y=rv_column_name,
                                 scatter=True, data=df, scatter_kws=scatter_kws)
xlabel_text = plt.xlabel('{} (explanatory variable)'.format(explanatory_variable))
ylabel_text = plt.ylabel('{} (response variable)'.format(response_variable))
kwargs = dict(textcoords='offset points', ha='left', va='bottom',
              bbox=dict(boxstyle='round,pad=0.5', fc='yellow', alpha=0.5),
              arrowprops=dict(arrowstyle='->', connectionstyle='arc3,rad=0'))
ev_max = df[ev_column_name].max()
ev_min = df[ev_column_name].min()
rv_min = df[rv_column_name].min()
rv_max = df[rv_column_name].max()
for label, x, y in zip(df.index, df[ev_column_name], df[rv_column_name]):
    if (x == ev_min):
        annotation = plt.annotate('{} ({})'.format(label, ev_min_str), xy=(x, y), xytext=ev_min_xytext, **kwargs)
    elif (x == ev_max):
        annotation = plt.annotate('{} ({})'.format(label, ev_max_str), xy=(x, y), xytext=ev_max_xytext, **kwargs)
    elif (y == rv_min):
        annotation = plt.annotate('{} ({})'.format(label, rv_min_str), xy=(x, y), xytext=rv_min_xytext, **kwargs)
    elif (y == rv_max):
        annotation = plt.annotate('{} ({})'.format(label, rv_max_str), xy=(x, y), xytext=rv_max_xytext, **kwargs)
    elif (y >= 36):
        annotation = plt.annotate('{} (pretty gun nutty)'.format(label), xy=(x, y), xytext=(-50, 50), **kwargs)
    elif (label == 'United States'):
        annotation = plt.annotate('{} (most evil)'.format(label), xy=(x, y), xytext=us_xytext, **kwargs)

In [ ]:

store_objects(RoL_df=RoL_df, capitalism_gini_df=capitalism_gini_df, gdp_df=gdp_df, governance_df=governance_df,
              gun_stats_df=gun_stats_df, income_df=income_df, merge_df=merge_df, typos_df=typos_df)

In [ ]:

dir(merge_axes_subplot)

In [ ]:

merge_axes_subplot.acorr?

In [ ]:

dir(merge_axes_subplot.collections[1])

In [ ]:

merge_axes_subplot.properties()

In [ ]:

RoL_dict

In [ ]:

%who

In [ ]:

RoL_df.columns

In [ ]:

columns_list = ['Series Name', 'Series Code']
values_list = RoL_df[columns_list].drop_duplicates().reset_index(drop=True).values
column_description_dict = {'_'.join(str(item[1]).split('.')):item[0] for item in values_list}
store_objects(RoL_dict=RoL_dict, column_description_dict=column_description_dict)